In [60]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.functional as F
import matplotlib.pyplot as plt

In [61]:
if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')


In [62]:
path = '../data/ml-1m/'

movies = pd.read_csv(path + 'movies.dat', sep = '::', engine = 'python', encoding = 'latin-1', names = ['movie_id', 'title', 'genres'])
ratings = pd.read_csv(path + 'ratings.dat', sep = '::', engine = 'python', encoding = 'latin-1', names = ['user_id', 'movie_id', 'rating', 'time'])
users = pd.read_csv(path + 'users.dat', sep = '::', engine = 'python', encoding = 'latin-1', names = ['user_id', 'gender', 'age', 'occupation', 'zipcode'])


In [63]:
movies.head()

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [64]:
movies['year'] = movies['title'].apply(lambda x: x.split('(')[1][:4])

In [65]:
pivot_matrix = ratings.pivot(index='user_id', columns='movie_id', values='rating')
original_rating_matrix = ratings.pivot(index='user_id', columns='movie_id', values='rating')

n_users, n_movies = pivot_matrix.values.shape
pivot_notna = pivot_matrix.notna()

In [66]:
import random
random.seed(42)
def train_test_split(matrix, ratio):
    true_indices = np.argwhere(matrix)

    num_test = int(len(true_indices) * ratio)
    np.random.shuffle(true_indices)

    test_indices = true_indices[:num_test]
    train_indices = true_indices[num_test:]

    train_dataset, test_dataset = np.zeros_like(matrix), np.zeros_like(matrix)

    for i in range(len(test_indices)):
        row_idx, column_idx = test_indices[i]
        test_dataset[row_idx, column_idx] = 1
    
    for i in range(len(train_indices)):
        row_idx, column_idx = train_indices[i]
        train_dataset[row_idx, column_idx] = 1

    return train_dataset, test_dataset

bin_train_data, bin_test_data = train_test_split(pivot_notna, 0.1)

In [67]:
def leave_one_out(matrix):
    n_rows, _ = matrix.shape
    train_indices, test_indices = [], []

    for i in range(n_rows):
        true_indices = np.argwhere(matrix[i, :])
        test_index = np.random.choice(true_indices.shape[0])
        test_indices.append([i,true_indices[test_index]])
        true_indices = np.delete(true_indices, test_index)
        row_train_indices = [[i, j] for j in true_indices.tolist()]
        train_indices += row_train_indices
        
    train_dataset, test_dataset = np.zeros_like(matrix), np.zeros_like(matrix)
    
    for i in range(len(test_indices)):
        row_idx, column_idx = test_indices[i]
        test_dataset[row_idx, column_idx] = 1
    
    for i in range(len(train_indices)):
        row_idx, column_idx = train_indices[i]
        train_dataset[row_idx, column_idx] = 1

    return train_dataset, test_dataset

loo_train_data, loo_test_data = leave_one_out(pivot_notna.values)

In [68]:
class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss,self).__init__()

    def forward(self, prediction, rating_matrix):
        non_zero_mask = (rating_matrix != -1).type(torch.FloatTensor)
        diff = (prediction - rating_matrix)**2
        prediction_error = (torch.sum(diff*non_zero_mask)/non_zero_mask.sum()) ** (1/2)
        return prediction_error.detach().numpy()
    
RMSE = RMSELoss()

In [82]:
class BPR(nn.Module):
    def __init__(self, n_users, n_items, dim, weight_decay):
        super().__init__()
        self.W = nn.Parameter(torch.empty(n_users, dim))
        self.H = nn.Parameter(torch.empty(n_items, dim))
        nn.init.xavier_normal_(self.W.data)
        nn.init.xavier_normal_(self.H.data)
        self.weight_decay = weight_decay

    def forward(self, u, i, j):
        u = self.W[u, :]
        i = self.H[i, :]
        j = self.H[j, :]
        x_ui = torch.matmul(u, i.T)
        x_uj = torch.matmul(u, j.T)
        x_uij = x_ui - x_uj
        log_prob = torch.sigmoid(x_uij).sum()
        regularization = self.weight_decay * (u.norm(p = 2) + i.norm(p = 2) + j.norm(p=2))
        return -log_prob + regularization

    def recommend(self, u):
        u = self.W[u, :]
        x_ui = torch.matmul(u, self.H.t())
        pred = torch.argsort(x_ui)
        return pred

In [83]:
bpr = BPR(n_users, n_movies, 10, 0.9)

In [84]:
bpr.recommend(0)

tensor([ 856, 3660, 3038,  ..., 1866,  762,  438])